In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/TimeLineChart.ipynb

Selectors :
   * absoluteToRelativePlayerProgress : checkboxe (T/F) value to display the line chart per relative time

In [ ]:
#constructing a checkboxe to view chart per relative time
absoluteToRelativePlayerProgress=widgets.Checkbox(value=False,
                                        description='View in relative time')

***vis_players_progress* function**:
* output : the outputPlayerProgress in a list to be display in a VBox
* outputPlayerProgress : the line chart showing progress over time for each player

In [1]:
def vis_players_progress():
    def display_player_progress(change):
        with outputPlayerProgress:
            clear_output(wait=False)
            players_progress()
            clear_output(wait=True)
    absoluteToRelativePlayerProgress.observe(display_player_progress, 'value')
    display_player_progress(None)
    return [absoluteToRelativePlayerProgress, outputPlayerProgressUpdate, outputPlayerProgress]

***vis_players_progress* function** displays line chart showing progress over time for each player

In [ ]:
def players_progress():
    #close and clear previous fig of progress players
    for fig in figPlayersProgress:
        plt.close(fig=fig)
    figPlayersProgress.clear()
    #get player selected list
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    empty=True
    #create new fig with custom name in the tab of player progress figs
    name=os.path.splitext(filename)[0]+"_players_progress"
    fig = plt.figure(num=name, constrained_layout=True)
    fig.canvas.header_visible = False
    fig.canvas.layout.min_height = '400px'
    figPlayersProgress.append(fig)
    #display a linechart for each player
    for player in players_info.keys():
        if player in playersSelected:
            if players_info[player]["game_progress_per_time"] != []:
                #list of time and progress value
                progress_time = list(zip(*players_info[player]["game_progress_per_time"]))
                y = [i*100 for i in progress_time[0]]
                if absoluteToRelativePlayerProgress.value:
                    x = absoluteToRelativeTime(progress_time[1])
                    plt.xlabel('Relative Time')
                else:
                    locator = mdates.AutoDateLocator()
                    plt.gca().xaxis.set_major_locator(locator)
                    plt.gca().xaxis.set_major_formatter(mdates.AutoDateFormatter(locator))
                    x =progress_time[1]
                    plt.xlabel('Time')
                plt.plot(x,y, label=player) # one line chart per player showing progress over time
                empty=False
    if not empty:
        #legend graph and give a date format
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.yticks(np.arange(0,101,10))
    else:
        #display not data to fill vis
        noDataToFillVis(100)
    #orient xticks
    plt.xticks(rotation=45, ha="right")
    #add title and axes labels
    if absoluteToRelativePlayerProgress.value:
        plt.xlabel('Relative Time')
    else:
        plt.xlabel('Time')
    plt.title("Progress (in %) of players by time")
    plt.ylabel('Progress (in %)')
    plt.show()
    with outputPlayerProgressUpdate:
        clear_output(wait=False)